In [44]:
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt

In [46]:
x,y = fetch_covtype(return_X_y=True)

In [19]:
#x.shape:(581012, 54)
#y.shape:(581012,)
#x_train.shape:(435759, 54) 25%
#x_test.shape:(145253, 54)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, shuffle=True, random_state=42)

In [42]:
xx_train, xx_test, yy_train, yy_test = train_test_split(x_test, y_test, test_size=0.25, shuffle=True, random_state=42)
print(xx_train.shape)
print(yy_test.shape)

(21788, 54)
(7263,)


In [43]:
k = range(0,len(xx_test))
error = []
for i in k:
    model = RandomForestClassifier()
    model.fit(xx_train, yy_train)
    prediction = model.predict(xx_test)
    error.append(np.mean(prediction != yy_test)*100)
print(error.shape)

KeyboardInterrupt: 

In [ ]:


# min_error = min(error)
# min_error_ind = error.index(min_error)
# k_opt = k[min_error_ind]
# print(f'Min error{k_opt}')
    
    
plt.plot(k,error)
# plt.plot(min_error,'ro', label = 'min error k')
plt.xlabel('k')
plt.ylabel('Error')
plt.show()

In [26]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from ray import tune
 
def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1
 
def training_function(config):
    # Hyperparameters
    alpha, beta = config["alpha"], config["beta"]
    for step in range(10):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(step, alpha, beta)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)
 
analysis = tune.run(
   training_function,
   config={
       "alpha": tune.grid_search([0.001, 0.01, 0.1]),
       "beta": tune.choice([1, 2, 3])
   })
 
print("Best config: ", analysis.get_best_config(
   metric="mean_loss", mode="min"))
 
# Get a dataframe for analyzing trial results.
df = analysis.results_df

In [33]:
#error_matrix[i,j] = (np.mean(prediction != Y_valid)*100) 

prediction = 2
Y_valid = 1
print(np.mean(prediction != Y_valid)*100)

100.0
